In [1]:
import pandas as pd
import os
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
app_csv = './data/apps_new.csv'
lib_csv = './data/libs_new.csv'

In [3]:
app_df = pd.read_csv(app_csv,sep='::')
lib_df = pd.read_csv(lib_csv,sep='::')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
import re
reg = r'[^0-9a-zA-Z]+'
reg= re.compile(reg)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict,Counter

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [5]:
def preprocess(texts,build_dic = True):
    stop_words = stopwords.words('english')

    # punctuations
    punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '\'']

    # tokenize
    # tokenizer = RegexpTokenizer(r'\w+')
    # raw_tokens = tokenizer.tokenize(doc.lower())
    # raw_tokens = word_tokenize(doc.lower())

    texts = word_tokenize(texts.lower())
    # texts = [
    #   word_tokenize(doc.lower())
    #   for doc in docs
    # ]

    # stop_words and punctuations
    texts = [w for w in texts if not w in stop_words+punctuations]
    # texts = [
      # [w for w in text if not w in stop_words + punctuations]
      # for text in texts
    # ]

    # remove words that appear only once
    if not hasattr(preprocess,'frequency'):
        preprocess.frequency = Counter()
        frequency =  preprocess.frequency
    else:
        frequency =  preprocess.frequency
        
    frequency.update(texts)
    # for text in texts:
    if build_dic:
        return 
    
    
    texts = [token for token in texts if frequency[token] >1]

    # word stem
    ps = PorterStemmer()
    # texts = [
    texts = [ps.stem(w) for w in texts]
      # for text in texts
    # ]
    if not texts:
        texts.append('None-Text')
    return texts

In [6]:
app_df['description'].apply(preprocess)


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
5244    None
5245    None
5246    None
5247    None
5248    None
5249    None
5250    None
5251    None
5252    None
5253    None
5254    None
5255    None
5256    None
5257    None
5258    None
5259    None
5260    None
5261    None
5262    None
5263    None
5264    None
5265    None
5266    None
5267    None
5268    None
5269    None
5270    None
5271    None
5272    None
5273    None
Name: description, Length: 5274, dtype: object

In [7]:
app_df['description'].apply(preprocess,build_dic = False)

0       [``, readi, learn, polish, welcom, babbel, dis...
1       [doctor, need, farm, look, littl, anim, look, ...
2       ['easi, fast, personalized\nfind, need, pagine...
3       [``, aaj, tak, india, 's, watch, news, channel...
4       [note, businfo, merg, businfong, becom, progra...
5       [``, walk, band, music, studio, toolkit, virtu...
6       [``, start, never, abl, stop, fall, love, conn...
7       ['矛掳赂矛\x97卢毛搂\x8c, 铆\x95麓毛\x8f\x84, 毛\xa0\x88矛...
8       ['asphalt, moto, 3d, motor, race, game, simpl,...
9       ['thi, game, romanian, language.\n\n, '', rasp...
10      [``, galaxi, parallax, live, wallpap, nebula, ...
11      ['use, desjardin, mobil, servic, make, standar...
12      ['googl, authent, gener, 2-step, verif, code, ...
13      [``, tuba.fm, free, music, app, smartphon, fin...
14      ['sticki, note, ultim, applic, creat, sticki, ...
15      ['blick, sport, 芒\x80\x93, die, sport-app, der...
16      [``, no.1, mobil, research, survey, applic, ov...
17      ['play

In [21]:
del preprocess.frequency 

In [6]:
def reprocess_df(df):
    df = df.set_index('kgid')
    df['description'].apply(preprocess)
    df['description'] = df['description'].apply(preprocess,build_dic = False)
    #df['description']= df['description'].apply(lambda x: reg.sub(" ",x))
    #df['description'] = df['description'].apply(str.split)
    return df

In [7]:
app_df = reprocess_df(app_df)
lib_df = reprocess_df(lib_df)


In [8]:
def load_lda_text(dataframe):
    texts = []
    for des in dataframe['description']:
        texts.append(des)
    return texts


In [9]:
app_text = load_lda_text(app_df)
lib_text = load_lda_text(lib_df)

In [10]:
lda_text = app_text #+ lib_text

In [11]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora.dictionary import Dictionary as gen_dict

In [14]:
lda_dict = gen_dict(lda_text)
lda_corpus = [lda_dict.doc2bow(text) for text in lda_text]
lda_model = LdaMulticore(lda_corpus,num_topics= 300,id2word=lda_dict)

In [15]:
model_path = r'./data/lda_datas/lda.model'
dict_path = r'./data/lda_datas/lda.dict'
lda_dict.save(dict_path)
lda_model.save(model_path)

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [30]:
app_df.to_hdf('./data/app_dfs',key = 'app')

In [29]:
lib_df.to_hdf('./data/lib_dfs',key = 'lib')

In [18]:
df = pd.concat([app_df,lib_df])

In [19]:
import pickle 

In [20]:
ui2index = pickle.load(open('./data/ui2index','rb'))

In [24]:
#df = df.set_index('kgid')
df = df.reset_index()

In [25]:
idx = df['kgid'].apply(lambda x: ui2index[str(x)])

In [26]:
df.index = idx

In [27]:
df.to_hdf('./data/id2text',key ='df')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['description']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [36]:
test = df.loc[5]['description']

In [39]:
co = lda_dict.doc2bow(test)

In [46]:
app_df.size

5274